# Информационный поиск
Скачиваем классический набор данных -- набор текстов об аэронавтике CRANFIELD

* `wget <option> <url>` - utility for downloading network data
* `tar <option> <archive_path>` - utility for creating and extracting files from archives

In [15]:
! wget -q http://ir.dcs.gla.ac.uk/resources/test_collections/cran/cran.tar.gz
! tar -xvf cran.tar.gz
! rm cran.tar.gz*

cran.all.1400
cran.qry
cranqrel
cranqrel.readme


Берём только сами запросы (это будут наши документы)

In [16]:
! grep -v "^\." cran.qry > just.qry
! head -3 just.qry

what similarity laws must be obeyed when constructing aeroelastic models
of heated high speed aircraft .
what are the structural and aeroelastic problems associated with flight


Объединяем многострочные в один

In [17]:
raw_query_data = [line.strip() for line in open("just.qry", "r").readlines()]
query_data = [""]

for query_part in raw_query_data:
  query_data[-1] += query_part + " "
  if query_part.endswith("."):
    query_data.append("")

query_data[:2] #Выведем пару документов для примера

['what similarity laws must be obeyed when constructing aeroelastic models of heated high speed aircraft . ',
 'what are the structural and aeroelastic problems associated with flight of high speed aircraft . ']

## Составим запросы к нашим документам

In [18]:
QUERIES = ['theory of bending', 'aeroelastic effects']

## Boolean retrieval

Представим каждый документ как "битовую маску": вектор размером со словарь, в котором на каждой позиции единица, если в документе есть соответствующий терм, и ноль, если терма нет.

Воспользуемся объектом [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)

In [19]:
# в разных версиях ответы могут отличаться, поэтому важно иметь одну и ту же
! pip install -q scikit-learn==0.22.2.post1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 45.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/pip/_internal/cli/base_command.py", line 160, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.9/dist-packages/pip/_internal/cli/req_command.py", line 241, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.9/dist-packages/pip/_internal/commands/install.py", line 463, in run
    _, build_failures = build(
  File "/usr/local/lib/python3.9/dist-packages/pip/_internal/wheel_builder.py", line 347, in build
    wheel_file = _build_one(
  File "/usr/local/lib/python3.9/dist-packages/pip/_internal/wheel_builder.py", line 221, in _build_one
    wheel_path = _build_one_inside_env(
  File "/usr/local/lib/python3.9/dist-packages/pip/_internal/wheel_builder.py", line 268, in _build_one_inside_env
    whe

In [20]:
from  sklearn.feature_extraction.text import CountVectorizer

encoder = CountVectorizer(binary=True)
encoded_data = encoder.fit_transform(query_data)
encoded_queries = encoder.transform(QUERIES)
list(encoder.vocabulary_)[:3]

['what', 'similarity', 'laws']

Посмотрим на представление первого предложения

In [21]:
id2term = {idx: term for term, idx in encoder.vocabulary_.items()}
non_zero_values_ids = encoded_data[0].nonzero()[1]

terms = [id2term[idx] for idx in non_zero_values_ids]
terms

['what',
 'similarity',
 'laws',
 'must',
 'be',
 'obeyed',
 'when',
 'constructing',
 'aeroelastic',
 'models',
 'of',
 'heated',
 'high',
 'speed',
 'aircraft']

Всё так.

## Задание 0

Теперь для каждого из данных запросов QUERIES найдём ближайший для него документ из query_data по сходству Жаккара. Есть более эффективные способы это сделать, но вам требуется реализовать расстояние Жаккара и далее применить его к нашим данным.

In [22]:
import numpy as np 

def jaccard_sim(vector_a: np.array, vector_b: np.array) -> float:
  """
    Сходство или коэффициент Жаккара: отношение мощности пересечения
    к мощности объединения
  """

  jaccard_dist = sum(vector_a & vector_b) / sum(vector_a | vector_b)
  return jaccard_dist

#Проверка, что функция работает правильно
assert jaccard_sim(np.array([1, 0, 1, 0, 1]), np.array([0, 1, 1, 1, 1])) == 0.4

Здесь документы представлены так же, как строки в матрице термов-документов. Каждая ячейка вектора отвечает за наличие/отсутствие конкретного элемента (например, слова-терма, когда у нас в словаре всего 5 слов). В первом случае их три, во втором — четыре. Объединение — все пять возможных элементов. Пересечение — два. Отсюда и 0.4.

## Задание 1

Теперь с помощью кода ниже вычислите для каждого запроса самые близкие документы.

In [23]:
for q_id, query in enumerate(encoded_queries):
  # приводим к нужному типу
  query = query.todense().A1
  docs = [doc.todense().A1 for doc in encoded_data]
  # вычисляем коэфф. Жаккара
  id2doc2similarity = [(doc_id, doc, jaccard_sim(query, doc)) for doc_id, doc in enumerate(docs)]
  # сортируем по нему
  closest = sorted(id2doc2similarity, key=lambda x: x[2], reverse=True)
  
  print("Q: %s\nFOUND:" % QUERIES[q_id])
  # выводим по 3 наиболее близких документа для каждого запроса
  for closest_id, _, sim in closest[:3]:
    print("    %d\t%.2f\t%s" %(closest_id, sim, query_data[closest_id]))

Q: theory of bending
FOUND:
    42	0.20	what are the details of the rigorous kinetic theory of gases . 
    43	0.20	(chapman-enskog theory) . 
    146	0.19	does a membrane theory exist by which the behaviour of pressurized membrane cylinders in bending can be predicted . 
Q: aeroelastic effects
FOUND:
    196	0.14	the problem of similarity for representative investigation of aeroelastic effects in a flow with the absence of heating effects . 
    204	0.12	do viscous effects seriously modify pressure distributions . 
    114	0.12	is the problem of similarity for representative investigations of aeroelastic effects in heated flow as intractable as previous investigations imply . 


Видим, что кое-где просачиваются тексты, которых с запросами объединяют малозначительные термы, но при этом коэффициент Жаккара -- наша функция ранжирования! -- высок.

## VSM

Попробуем теперь сделать то же, но с tf-idf и косинусным расстоянием. Мы сделаем всё опять "руками", но "в реальной жизни" лучше использоватьесть эффективные реализации cosine distance, например, из библиотеки scipy.

In [24]:
from  sklearn.feature_extraction.text import TfidfVectorizer

# Совет: обязательно разберитесь с тем, какие возможности
# предоставляет tf-idf vectorizer, какие параметры за что отвечают

tfidf_encoder = TfidfVectorizer()
tfidf_encoded_data = tfidf_encoder.fit_transform(query_data)
tfidf_encoded_queries = tfidf_encoder.transform(QUERIES)

list(tfidf_encoder.vocabulary_)[:3]

['what', 'similarity', 'laws']

## Задание 2

Реализовать косинусное расстояние

Скалярным произведением двух векторов называется число, равное произведению длин этих векторов на косинус угла между ними.

Норма вектора - это длина вектора.

In [29]:
import numpy as np 

def cosine_distance(vector_a: np.array, vector_b: np.array) -> float:
  """
    Косинусное расстояние: единица минус отношение скалярного произведения
    на произведение L2-норм (подсказка: в numpy такие нормы есть)
  """

  return 1 - np.dot(vector_a, vector_b) / (np.linalg.norm(vector_a) * np.linalg.norm(vector_b))

#Проверка, что функция работает правильно
assert cosine_distance(np.array([1, 0, 1, 1, 1]), np.array([0, 0, 1, 0, 0])) == 0.5

Теперь вычислим ближайшие по косинусному расстоянию между векторными представлениями документов и запросов

In [30]:
for q_id, query in enumerate(tfidf_encoded_queries):
  
  # приводим к нужному типу
  query = query.todense().A1
  docs = [doc.todense().A1 for doc in tfidf_encoded_data]
  # Косинусное расстояние
  id2doc2similarity = [(doc_id, doc, cosine_distance(query, doc)) \
                       for doc_id, doc in enumerate(docs)]
  # сортируем по нему
  closest = sorted(id2doc2similarity, key=lambda x: x[2], reverse=False)
  
  print("Q: %s\nFOUND:" % QUERIES[q_id])
  
  for closest_id, _, sim in closest[:3]:
    print("    %d\t%.2f\t%s" %(closest_id, sim, query_data[closest_id]))

Q: theory of bending
FOUND:
    145	0.65	what are the best experimental data and classical small deflection theory analyses available for pressurized cylinders in bending . 
    146	0.66	does a membrane theory exist by which the behaviour of pressurized membrane cylinders in bending can be predicted . 
    111	0.66	has the solution of the clamped plate problem,  in the classical theory of bending,  been reduced to two successive membrane boundary value problems . 
Q: aeroelastic effects
FOUND:
    196	0.51	the problem of similarity for representative investigation of aeroelastic effects in a flow with the absence of heating effects . 
    114	0.70	is the problem of similarity for representative investigations of aeroelastic effects in heated flow as intractable as previous investigations imply . 
    1	0.74	what are the structural and aeroelastic problems associated with flight of high speed aircraft . 


<ipython-input-29-b0763db6b950>:9: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - np.dot(vector_a, vector_b) / (np.linalg.norm(vector_a) * np.linalg.norm(vector_b))
